# **Function Root Estimator**
## **Contents**
1. [Introduction](#Introduction)
2. [The Newton-Raphson Method](#Brownian-Motion-Simulator)
3. [The Secant Method](#Brownian-Motion-Parameter-Dependent-Simulator)
4. [The Bisection Method](#Geometric-Brownian-Motion-Simulator)
5. [Credits](#Credits)

In [209]:
import numpy as np
from time import time

def performance_metrics(*args, method):
    start = time()
    x = method(*args)
    end = time()
    s = round(end - start, 3)
    f = args[0]
    n = args[-2]
    d = args[-1]
    print(f"Function: {f}", f"Method: {method.__name__}", f"Result: {x}", f"Iteration limit: {n}", f"Precision: {d}", f"Computation time: {s} seconds.", sep = "\n")

In [213]:
def newton_raphson(f, df, x0, n = 100, d = 1e-6):
    f = np.vectorize(f)
    df = np.vectorize(df)
    x0 = np.array(x0)
    assert type(n) is int and n >= 1 and np.all(df(x0) != 0)
    for _ in range(n):
        x1 = np.subtract(x0, np.divide(f(x0), df(x0)))
        if np.all(np.abs(f(x1)) < d):
            break
        else:
            pass
        x0 = x1
    return np.array([x1, f(x1)])

performance_metrics(lambda x: x**3, lambda x: 3 * x**2, [1, 2, 3], 100, 1e-6, method = newton_raphson)

Function: <function <lambda> at 0x00000215401F37F0>
Method: newton_raphson
Result: [[2.28365826e-03 4.56731652e-03 6.85097478e-03]
 [1.19094949e-08 9.52759591e-08 3.21556362e-07]]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.001 seconds.


In [214]:
def secant(f, x0, x1, n = 100, d = 1e-6):
    f = np.vectorize(f)
    x0 = np.array(x0)
    x1 = np.array(x1)
    assert type(n) is int and n >= 1 and np.all(f(x0) != f(x1))
    for _ in range(n):
        x2 = np.divide(np.subtract(np.multiply(x0, f(x1)), np.multiply(x1, f(x0))), np.subtract(f(x1), f(x0)))
        if np.all(np.abs(f(x2)) < d):
            break
        else:
            pass
        x0 = x1
        x1 = x2
    return np.array([x2, f(x2)])

performance_metrics(lambda x: x**3, [1,2], [3,4], 100, 1e-6, method = secant)

Function: <function <lambda> at 0x000002154016B130>
Method: secant
Result: [[5.10209540e-03 9.16331840e-03]
 [1.32814572e-07 7.69410897e-07]]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.002 seconds.


In [215]:
def bisection(f, a, b, n = 100, d = 1e-6):
    f = np.vectorize(f)
    a = np.array(a)
    b = np.array(b)
    assert type(n) is int and n >= 1 and np.all(a < b and f(a) * f(b) <= 0)
    for _ in range(n):
        c = np.divide(np.add(a, b), 2)
        if np.all(np.abs(f(c)) < d):
            break
        elif np.any(f(c) < -d):
            a = c
        else:
            b = c
    return np.array([c, f(c)])

print(performance_metrics(lambda x: x**3, -1, 1, 100, 1e-6, method = bisection))

Function: <function <lambda> at 0x000002154016AA70>
Method: bisection
Result: [0. 0.]
Iteration limit: 100
Precision: 1e-06
Computation time: 0.0 seconds.
None
